In [1]:
import os
import math

import pandas as pd
import numpy as np

import sys
sys.path.append("..") # Adds higher directory to python modules path.
from Jetson.autopilot.record.AutopilotGUI import AutopilotGUI

In [2]:
datasets_path = "../notebooks/datasets/validation/"
sub_dirs = os.listdir(datasets_path)
dataset_paths = [[datasets_path + sub_dir + "/" + sub_dir + ".csv", datasets_path + sub_dir + "/frames/"] for sub_dir in sub_dirs]
dataset_paths

[['../notebooks/datasets/validation/Tag-Sonne-Wolken-Ausserorts/Tag-Sonne-Wolken-Ausserorts.csv',
  '../notebooks/datasets/validation/Tag-Sonne-Wolken-Ausserorts/frames/'],
 ['../notebooks/datasets/validation/Nacht-Schneechaos/Nacht-Schneechaos.csv',
  '../notebooks/datasets/validation/Nacht-Schneechaos/frames/']]

In [3]:
from RoadDataset import RoadDataset

In [4]:
dataset_index = 0
column_names = ['filename','steering_wheel_angle','speed', 'blink_l', 'blink_r']
dataset = RoadDataset(dataset_paths[dataset_index][0], dataset_paths[dataset_index][1], column_names, mode='train')

Loaded dataset with 10474 samples


In [5]:
#dataset.pairplot()

In [6]:
dataset.clean()

Cleaning dataset. Starting with 10474 samples.
Removing 1205 rows for reason: Low speed (< 25 km/h)
Removing 541 rows for reason: High SWA (> +45/ < -45 deg)
Removing 66 rows for reason: Left blinker on
Removing 168 rows for reason: Right blinker on
8494 samples remaining.


In [7]:
#dataset.pairplot()

In [8]:
dataset.balance()

Balancing dataset. Starting with 8494 samples.
2281 samples remaining.


In [9]:
dataset.normalize()

In [10]:
#dataset.pairplot()

In [11]:
gui = AutopilotGUI()

In [12]:
from tensorflow.keras.preprocessing.image import img_to_array

In [13]:
def show_frame(index):
    
    frame = img_to_array(dataset.get_frame(index)[0])
    gui.set_frame(frame)
    gui.set_show_overlay(True)
    gui.set_engaged(False)
    gui.set_recording(True)
    
    blink_l = dataset.get_indicator_left(index)
    print("blink_l: " + str(blink_l))
    gui.set_indicator_left(blink_l)
    #gui.set_indicator_right(indicator_right)
    #gui.set_cruise_control(cruise_control)
    #gui.set_cruise_control_setpoint(cc_setpoint)
    
    velocity = dataset.get_velocity(index)
    gui.set_velocity(velocity)
    
    timestamp = dataset.get_timestamp(index)
    gui.set_timestring(timestamp)
    
    
    actual_swa = dataset.get_swa(index)*90
    print("actual_swa: " + str(actual_swa))
    gui.set_actual_swa(actual_swa)
    #gui.set_predicted_swa(predicted_swa)
    
    gui.show_jupyter()
    
    
from matplotlib import pyplot as plt
    
def show_crop(index, shift):
    frame = img_to_array(dataset.get_cropped_frame(index, shift)[0])
    plt.imshow(frame/255)
    plt.show()

In [14]:
import ipywidgets as widgets
frame_slider = widgets.IntSlider(min=0, max=dataset.get_batch_size()*len(dataset)-1, continuous_update=False)

In [15]:
from ipywidgets import interact

interact(show_frame, index=frame_slider);

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=2271), Output()), _…

In [16]:
crop_slider = widgets.IntSlider(min=0, max=dataset.get_batch_size()*len(dataset)-1, continuous_update=False)
shift_slider = widgets.IntSlider(min=-100, max=100, continuous_update=False)

In [17]:
interact(show_crop, index=crop_slider, shift = shift_slider);

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=2271), IntSlider(va…

In [18]:
train_stats = dataset.get_csv().describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
steering_wheel_angle,2281.0,-0.003996,0.214219,-0.498111,-0.176,-0.000667,0.167667,0.480667
speed,2281.0,51.497224,11.559203,25.203000,43.648,50.840000,58.528000,77.066000
blink_l,2281.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
blink_r,2281.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000


In [19]:
dataset.get_batch_labels(0)

array([-0.02088889,  0.08277778,  0.09044444,  0.09044444,  0.09044444,
        0.09044444,  0.08488889,  0.06677778,  0.06188889,  0.05911111,
       -0.15933333, -0.15933333, -0.15933333, -0.16144444, -0.16144444,
       -0.16144444])

In [20]:
feature_batch = dataset.get_batch_features(0)
feature_batch

array([[[[104.,  82.,  59.],
         [102.,  80.,  57.],
         [102.,  82.,  58.],
         ...,
         [ 90.,  77.,  61.],
         [ 93.,  77.,  64.],
         [ 99.,  83.,  70.]],

        [[ 93.,  69.,  45.],
         [101.,  76.,  54.],
         [ 90.,  68.,  45.],
         ...,
         [100.,  86.,  73.],
         [109.,  93.,  80.],
         [ 99.,  83.,  70.]],

        [[ 82.,  66.,  40.],
         [ 86.,  70.,  44.],
         [ 84.,  68.,  43.],
         ...,
         [104.,  95.,  90.],
         [126., 117., 112.],
         [147., 139., 136.]],

        ...,

        [[ 37.,  47.,  38.],
         [ 39.,  49.,  40.],
         [ 40.,  50.,  41.],
         ...,
         [ 80.,  78.,  57.],
         [ 81.,  79.,  58.],
         [ 80.,  78.,  57.]],

        [[ 37.,  47.,  38.],
         [ 38.,  48.,  39.],
         [ 40.,  50.,  41.],
         ...,
         [ 80.,  78.,  57.],
         [ 82.,  80.,  59.],
         [ 83.,  81.,  60.]],

        [[ 37.,  47.,  38.],
       

In [21]:
batch_count = len(dataset)
batch_count

142